In [ ]:
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)
import numpy as np
print(np.__version__)

from keras_tqdm import TQDMNotebookCallback


from keras.applications.densenet import DenseNet201
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

from keras.optimizers import Adam

In [ ]:
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.summary()

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# define metric
from balancedAccuracy import balancedAccuracy
num_classes = 3
bacc_metric = balancedAccuracy(num_classes)

In [ ]:
# plot_confusion_matrix function
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

class_names = ["MEL", "NV", "BKL"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(np.mean(np.diag(cm)))
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    # get balanced accuracy
    return np.mean(np.diag(cm))
    


In [ ]:
data = np.load(".../HAMAUG.npz")

imageList = data["imageList"]
targetList = data["targetList"]
imageValList = data["imageValList"]
targetValList = data["targetValList"]

targetList = targetList[:,:3]
targetValList= targetValList[:,:3]

In [ ]:
testdata = np.load(".../TESTHAM.npz")
testList = testdata["testList"]
targetTestList = testdata["targetTestList"]

In [ ]:
def valMatrix():
    y_test = targetValList.copy()
    y_pred = model.predict(imageValList)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Validation confusion matrix')))
    plt.show()

def trainMatrix():
    y_test = targetListUltra.copy()
    y_pred = model.predict(imageListUltra)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Training confusion matrix')))
    plt.show()
    
def testMatrix():
    y_test = targetTestList.copy()
    y_pred = model.predict(testList)
    y_pred = y_pred.argmax(1)
    y_test = y_test.argmax(1)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    plt.figure()
    print("Balanced Accuracy: "+ str(plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized Training confusion matrix')))
    plt.show()

## Additional Data Augmentation with AC-GAN

In [ ]:
targetList.sum(axis=0)/5, targetTestList.sum(axis=0)

In [ ]:
sumos = targetList.sum(axis=0)/5 # /5 to account for traditional augmentation
GAUG_FACTOR = float(input()) # varying this value varies the level of GAN-augmentation

GAUG_LVL = np.round(sumos*GAUG_FACTOR).astype(int) # for use in generation
print(GAUG_LVL)

In [ ]:
from keras.models import model_from_json, load_model
import json
import cv2 # pip install opencv-python

json_file = open('ACGAN-weights/generator.json', 'r')
generator_json = json_file.read()
json_file.close()
generator = model_from_json(generator_json)
generator.load_weights('ACGAN-weights/generator_weights.hdf5')

In [ ]:
noise = np.random.normal(0, 1, (GAUG_LVL[0], 128))
sampled_labels_MEL = np.zeros((GAUG_LVL[0],))
                          
MEL_64 = 0.5*generator.predict([noise, sampled_labels_MEL]) + 0.5

MEL = []

for i in range(MEL_64.shape[0]):
    res = cv2.resize(MEL_64[i], dsize=(224, 224), interpolation=cv2.INTER_LANCZOS4)
    MEL.append(res)

In [ ]:
MEL = np.array(MEL)
print(MEL.shape)

In [ ]:
noise = np.random.normal(0, 1, (GAUG_LVL[1], 128))
sampled_labels_NEV = np.zeros((GAUG_LVL[1],)) + 1
                          
NEV_64 = 0.5*generator.predict([noise, sampled_labels_NEV]) + 0.5

NEV = []

for i in range(NEV_64.shape[0]):
    res = cv2.resize(NEV_64[i], dsize=(224, 224), interpolation=cv2.INTER_LANCZOS4)
    NEV.append(res)

In [ ]:
NEV = np.array(NEV)
print(NEV.shape)

In [ ]:
noise = np.random.normal(0, 1, (GAUG_LVL[2], 128))
sampled_labels_BKL = np.zeros((GAUG_LVL[2],)) + 2
                          
BKL_64 = 0.5*generator.predict([noise, sampled_labels_BKL]) + 0.5

BKL = []

for i in range(BKL_64.shape[0]):
    res = cv2.resize(BKL_64[i], dsize=(224, 224), interpolation=cv2.INTER_LANCZOS4)
    BKL.append(res)

In [ ]:
BKL = np.array(BKL)
print(BKL.shape)

In [ ]:
imageListUltra = np.concatenate([imageList, MEL,NEV,BKL])

In [ ]:
targetListUltra = np.concatenate([targetList.argmax(axis=1), sampled_labels_MEL,sampled_labels_NEV,sampled_labels_BKL])

## TRAIN

In [ ]:
# get sample weights for training (sk's class_weight doesn't work with one-hot encoding)
weight_dict = {0:8.15, 1:1.33, 2:8.08}
sample_weights = np.array(list(map(weight_dict.get, targetListUltra)))

# get sample weights for validation
temp = targetValList.argmax(1)
sample_weights_val = np.array(list(map(weight_dict.get, temp)))

In [ ]:
from keras.applications.densenet import preprocess_input
imageListUltra = preprocess_input(imageListUltra)
imageValList = preprocess_input(imageValList)
testList = preprocess_input(testList)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

targetListUltra = OneHotEncoder().fit_transform(targetListUltra.reshape(-1,1))

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
              metrics=[bacc_metric.balanced_acc])
historyfile = model.fit(imageListUltra, targetListUltra, batch_size=32, epochs=1, verbose=0,
          sample_weight=sample_weights,
          validation_data=(imageValList, targetValList, sample_weights_val), initial_epoch=0,
          callbacks = [TQDMNotebookCallback(leave_inner=True)])


In [ ]:
valMatrix()

In [ ]:
testMatrix()

In [ ]:
model.compile(optimizer=Adam(lr=0.00003), loss='categorical_crossentropy',
              metrics=[bacc_metric.balanced_acc])
historyfile = model.fit(imageListUltra, targetListUltra, batch_size=32, epochs=1, verbose=0,
          sample_weight=sample_weights,
          validation_data=(imageValList, targetValList, sample_weights_val), initial_epoch=0,
          callbacks = [TQDMNotebookCallback(leave_inner=True)])


In [ ]:
valMatrix()

In [ ]:
testMatrix()

In [ ]:
model.save("densenet201.h5")